## Imports

In [1]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from collections import Counter
from num2words import num2words
import nltk
import os
import string
import numpy as np
import copy
import pandas as pd
import pickle
import re
import math

/Users/sharan/anaconda3/lib/python3.6/site-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/Users/sharan/.matplotlib/matplotlibrc", line #39
  (fname, cnt))


In [2]:
df = pd.read_csv('dataset.csv')

In [3]:
def print_doc(id):
    print(dataset[id])
    file = open(dataset[id][0], 'r', encoding='cp1250')
    text = file.read().strip()
    file.close()
    print(text)

# Preprocessing

In [4]:
def convert_lower_case(data):
    return np.char.lower(data)

In [5]:
def remove_stop_words(data):
    stop_words = stopwords.words('english')
    words = word_tokenize(str(data))
    new_text = ""
    for w in words:
        if w not in stop_words and len(w) > 1:
            new_text = new_text + " " + w
    return new_text

In [6]:
def remove_punctuation(data):
    symbols = "!\"#$%&()*+-./:;<=>?@[\]^_`{|}~\n"
    for i in range(len(symbols)):
        data = np.char.replace(data, symbols[i], ' ')
        data = np.char.replace(data, "  ", " ")
    data = np.char.replace(data, ',', '')
    return data

In [7]:
def remove_apostrophe(data):
    return np.char.replace(data, "'", "")

In [8]:
def stemming(data):
    stemmer= PorterStemmer()
    
    tokens = word_tokenize(str(data))
    new_text = ""
    for w in tokens:
        new_text = new_text + " " + stemmer.stem(w)
    return new_text

In [9]:
def convert_numbers(data):
    tokens = word_tokenize(str(data))
    new_text = ""
    for w in tokens:
        try:
            w = num2words(int(w))
        except:
            a = 0
        new_text = new_text + " " + w
    new_text = np.char.replace(new_text, "-", " ")
    return new_text

In [10]:
def preprocess(data):
    data = convert_lower_case(data)
    data = remove_punctuation(data) #remove comma seperately
    data = remove_apostrophe(data)
    data = remove_stop_words(data)
    data = convert_numbers(data)
    data = stemming(data)
    data = remove_punctuation(data)
    data = convert_numbers(data)
    data = stemming(data) #needed again as we need to stem the words
    data = remove_punctuation(data) #needed again as num2word is giving few hypens and commas fourty-one
    data = remove_stop_words(data) #needed again as num2word is giving stop words 101 - one hundred and one
    return data

## Extracting Data

In [11]:
processed_text = []
processed_title = []
texts = []

for row in df.iterrows():
    processed_text.append(word_tokenize(str(preprocess(row[1]['text']))))
    processed_title.append(word_tokenize(str(preprocess(row[1]['title']))))

KeyError: 'title'

### Calculating document frequency for all words

In [12]:
doc_freq = {}
for i in range(len(dataset)):
    tokens = processed_text[i]
    for w in tokens:
        try:
            doc_freq[w].add(i)
        except:
            doc_freq[w] = {i}

    tokens = processed_title[i]
    for w in tokens:
        try:
            doc_freq[w].add(i)
        except:
            doc_freq[w] = {i}
for i in doc_freq:
    doc_freq[i] = len(doc_freq[i])

NameError: name 'dataset' is not defined

In [23]:
total_vocab_size = len(DF)

In [24]:
total_vocab_size

32350

In [25]:
total_vocab = [x for x in DF]

In [26]:
print(total_vocab[:20])

['sharewar', 'trial', 'project', 'freewar', 'need', 'support', 'continu', 'one', 'hundr', 'west', 'fifti', 'three', 'north', 'jim', 'prentic', 'copyright', 'thousand', 'nine', 'nineti', 'brandon']


In [27]:
def doc_freq(word):
    c = 0
    try:
        c = DF[word]
    except:
        pass
    return c

### Calculating TF-IDF for body, we will consider this as the actual tf-idf as we will add the title weight to this.

In [28]:
doc = 0

tf_idf = {}

for i in range(N):
    
    tokens = processed_text[i]
    
    counter = Counter(tokens + processed_title[i])
    words_count = len(tokens + processed_title[i])
    
    for token in np.unique(tokens):
        
        tf = counter[token]/words_count
        df = doc_freq(token)
        idf = np.log((N+1)/(df+1))
        
        tf_idf[doc, token] = tf*idf

    doc += 1

In [26]:
# tf_idf

### Calculating TF-IDF for Title

In [29]:
doc = 0

tf_idf_title = {}

for i in range(N):
    
    tokens = processed_title[i]
    counter = Counter(tokens + processed_text[i])
    words_count = len(tokens + processed_text[i])

    for token in np.unique(tokens):
        
        tf = counter[token]/words_count
        df = doc_freq(token)
        idf = np.log((N+1)/(df+1)) #numerator is added 1 to avoid negative values
        
        tf_idf_title[doc, token] = tf*idf

    doc += 1

In [30]:
# tf_idf_title

In [31]:
tf_idf[(0,"go")]

0.0002906893990853149

In [32]:
tf_idf_title[(0,"go")]

0.0002906893990853149

## Merging the TF-IDF according to weights

In [33]:
alpha = 0.3
for i in tf_idf:
    tf_idf[i] *= alpha

In [34]:
for i in tf_idf_title:
    tf_idf[i] = tf_idf_title[i]

In [35]:
len(tf_idf)

344378

# TF-IDF Matching Score Ranking

In [37]:
def matching_score(k, query):
    preprocessed_query = preprocess(query)
    tokens = word_tokenize(str(preprocessed_query))

    print("Matching Score")
    print("\nQuery:", query)
    print("")
    print(tokens)
    
    query_weights = {}

    for key in tf_idf:
        
        if key[1] in tokens:
            try:
                query_weights[key[0]] += tf_idf[key]
            except:
                query_weights[key[0]] = tf_idf[key]
    
    query_weights = sorted(query_weights.items(), key=lambda x: x[1], reverse=True)

    print("")
    
    l = []
    
    for i in query_weights[:10]:
        l.append(i[0])
    
    print(l)
    

matching_score(10, "Horror")

Matching Score

Query: Horror

['horror']

[102, 409, 100, 69, 138, 139, 12, 302, 62, 349]


In [41]:
print_doc(69)

('/Users/sharan/aarhus_itk/Information-Retrieval/2. TF-IDF Ranking - Cosine Similarity, Matching Score/stories/bluebrd.txt', 'Bluebeard')
BLUEBEARD

   Once upon a time... in the fair land of France, there lived a very powerful 
lord, the owner of estates, farms and a great splendid castle, and his name was
Bluebeard. This wasn't his real name, it was a nickname, due to the fact he had
a long shaggy black beard with glints of blue in it. He was very handsome and
charming, but, if the truth be told, there was something about him that made 
you feel respect, and a little uneasy...
   Bluebeard often went away to war, and when he did, he left his wife in 
charge of the castle... He had had lots of wives, all young, pretty and noble.
As bad luck would have it, one after the other, they had all died, and so the
noble lord was forever getting married again.
   "Sire," someone would ask now and again, "what did your wives die of?"
   "Hah, my friend," Bluebeard would reply, "one died of small

# TF-IDF Cosine Similarity Ranking

In [42]:
def cosine_sim(a, b):
    cos_sim = np.dot(a, b)/(np.linalg.norm(a)*np.linalg.norm(b))
    return cos_sim

### Vectorising tf-idf

In [43]:
D = np.zeros((N, total_vocab_size))
for i in tf_idf:
    try:
        ind = total_vocab.index(i[1])
        D[i[0]][ind] = tf_idf[i]
    except:
        pass

In [44]:
def gen_vector(tokens):

    Q = np.zeros((len(total_vocab)))
    
    counter = Counter(tokens)
    words_count = len(tokens)

    query_weights = {}
    
    for token in np.unique(tokens):
        
        tf = counter[token]/words_count
        df = doc_freq(token)
        idf = math.log((N+1)/(df+1))

        try:
            ind = total_vocab.index(token)
            Q[ind] = tf*idf
        except:
            pass
    return Q

In [49]:
def cosine_similarity(k, query):
    print("Cosine Similarity")
    preprocessed_query = preprocess(query)
    tokens = word_tokenize(str(preprocessed_query))
    
    print("\nQuery:", query)
    print("")
    print(tokens)
    
    d_cosines = []
    
    query_vector = gen_vector(tokens)
    
    for d in D:
        d_cosines.append(cosine_sim(query_vector, d))
        
    out = np.array(d_cosines).argsort()[-k:][::-1]
    
    print("")
    
    print(out)

#     for i in out:
#         print(i, dataset[i][0])

Q = cosine_similarity(10, "Adventure")

Cosine Similarity

Query: 51

['fifti', 'one']

[  0 101 167 211 127 307  44 154  87 438]


In [50]:
print_doc(51)

('/Users/sharan/aarhus_itk/Information-Retrieval/2. TF-IDF Ranking - Cosine Similarity, Matching Score/stories/bern', 'The Adventures of Bert and Bernece, by  Francis U. Kaltenbaugh')
THE ADVENTURES OF BERT AND BERNECE
  by Francis U. Kaltenbaugh

  In mid-town, the sun's brazen harshness was reinforced, as
it glared from a glass and ivory colored office building towering 
towards the heavens, stiff and erect in stature; symbolism oozed 
from its solar-heated shaft, as an unnoticed conversation unfolded 
ensconced near the tip of this man-made erection of glass and steel.
  
  "Stop squirming. You'll die for what you did," Bert threatened.
  
  "You'll never get away with this," I lied. "There are others, who 
know I came here for you."
  
  "You stole my woman; you're gonna pay," Bert accused.
  
  "What woman? I don't have a woman -- not me. I'm to enter seminary 
next month. I'm celibate," I babbled.
  
  "Sell a bit!  What the hell ... a polite way to say pimp or 
whoremaster?" he 